参考：  https://towardsdatascience.com/train-an-object-detector-using-tensorflow-2-object-detection-api-in-2021-a4fed450d1b9

# 事前準備
ランタイム→ランタイムのタイプを変更を選択、ハードウェアアクセラレータを「GPU」に変更しておく。


# ターゲットの指定
HAND: 手の認識  
それ以外：Pascal VOC  


In [ ]:
# TARGET_DATA="HAND"
TARGET_DATA="VOC"
print(TARGET_DATA)


# GoogleDriveのマウント
中断後の処理再開に必要なファイルをGoogleDrive上にファイルを保存するため、GoogleDriveをマウントする。

In [ ]:
import sys
import os

# ベースディレクトリ
BASE_DIR = "/content"

if TARGET_DATA == "HAND" :
  # ワークディレクトリ
  WORK_DIR = '/content/drive/MyDrive/hand_detect'
else :
  # ワークディレクトリ
  WORK_DIR = '/content/drive/MyDrive/voc_detect'


from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

if os.path.exists(WORK_DIR) :
  # 2回目以降の実行
  FIRST_EXEC=False
  print("2nd ececution")
else :
  # 最初の実行
  FIRST_EXEC=True
  !mkdir -p $WORK_DIR
  print("1st ececution")


# object-detection モジュールのインストール
## gitリポジトリのclone ～ インストール

In [ ]:
%cd $BASE_DIR
!git clone --depth 1 https://github.com/tensorflow/models.git

# プロトコルバッファのコンパイル
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

# モジュールのインストール
!cp object_detection/packages/tf2/setup.py . 
!python -m pip install .


## テスト

In [ ]:
%cd $BASE_DIR/models/research
!python object_detection/builders/model_builder_tf2_test.py

# データセットのダウンロード
## gitリポジトリのclone→csvファイル→tf_recorfファイル

In [ ]:
LABEL_MAP = os.path.join(WORK_DIR, "label_map.pbtxt")
TRAIN_FILE = os.path.join(WORK_DIR, "train.record")
TEST_FILE  = os.path.join(WORK_DIR, "test.record")

if TARGET_DATA == "HAND" :

  if FIRST_EXEC :
    %cd $BASE_DIR
    !git clone https://github.com/aalpatya/detect_hands.git
    !python detect_hands/egohands_dataset_to_csv.py
    !python detect_hands/generate_tfrecord.py --csv_input=images/train/train_labels.csv  --output_path=$TRAIN_FILE
    !python detect_hands/generate_tfrecord.py --csv_input=images/test/test_labels.csv    --output_path=$TEST_FILE

    # ラベルファイルをコピー
    ! cp ./detect_hands/model_data/ssd_mobilenet_v2_fpn_320/label_map.pbtxt $LABEL_MAP
  else :
      print("2nd ececution")

else :
  if FIRST_EXEC :
    %cd $BASE_DIR
    # !wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar -O - | tar xvf -
    # ミラーサイト使用の場合はこちら
    !wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar  -O - | tar xvf -
    %cd $BASE_DIR/models/research/object_detection/
    !cp ./data/pascal_label_map.pbtxt $LABEL_MAP
    !python dataset_tools/create_pascal_tf_record.py --label_map_path $LABEL_MAP --data_dir $BASE_DIR/VOCdevkit --year VOC2007 --set train --output_path $TRAIN_FILE
    !python dataset_tools/create_pascal_tf_record.py --label_map_path $LABEL_MAP --data_dir $BASE_DIR/VOCdevkit --year VOC2007 --set val   --output_path $TEST_FILE

  else :
      print("2nd ececution")


# 元となるモデルのダウンロード

元になるモデルファイルは以下を参照  
https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md


In [ ]:
# 学習済みモデル出力ディレクトリ
TRAINED_DIR_REL = "output_training"
TRAINED_DIR = os.path.join(WORK_DIR, TRAINED_DIR_REL)

# CONFIGファイル名
CONFIG_FILE     = os.path.join(WORK_DIR, "pipeline.config")

if FIRST_EXEC :
  %cd $WORK_DIR
  
  # 元となるモデルのダウンロード
  !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz -O - | tar xzvf -

  BASE_MODEL_DIR = os.path.join(WORK_DIR, "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8")
  # 元となるモデルのCHECKPOINTファイル
  CKPT_FILE   =  os.path.join(BASE_MODEL_DIR, "checkpoint/ckpt-0")
  # 元となるモデルのconfigファイル
  CONFIG_FILE_ORG = os.path.join(BASE_MODEL_DIR, "pipeline.config")

  # 作業用CONFIGファイルを作成
  !cp $CONFIG_FILE_ORG $CONFIG_FILE

  if TARGET_DATA == "HAND" :
    # 変更パラメータ
    NUM_CLASSES = 1                   # クラス数
    NUM_STEPS = 50000                 # 回数は要検討
    BATCH_SIZE = 4                    # バッチサイズ
  else :
    # 変更パラメータ
    NUM_CLASSES = 20                  # クラス数
    NUM_STEPS = 10000                 # 回数は要検討
    BATCH_SIZE = 64                   # バッチサイズ

  from object_detection.protos import pipeline_pb2
  from google.protobuf import text_format
  import tensorflow.compat.v1 as tf

  # CONFIGファイル読み込み
  pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
  with tf.gfile.GFile(CONFIG_FILE, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

  # パラメータ変更
  pipeline_config.model.ssd.num_classes                                      = NUM_CLASSES

  pipeline_config.train_config.batch_size                                    = BATCH_SIZE
  pipeline_config.train_config.fine_tune_checkpoint                          = CKPT_FILE
  pipeline_config.train_config.num_steps                                     = NUM_STEPS
  pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.total_steps = NUM_STEPS
  pipeline_config.train_config.fine_tune_checkpoint_type                     = "detection"

  pipeline_config.train_input_reader.label_map_path                          = LABEL_MAP
  pipeline_config.train_input_reader.tf_record_input_reader.input_path[0]    = TRAIN_FILE

  pipeline_config.eval_input_reader[0].label_map_path                        = LABEL_MAP
  pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0]  = TEST_FILE

  # 変更後データの書き込み
  pipeline_text = text_format.MessageToString(pipeline_config)
  with tf.gfile.Open(CONFIG_FILE, "wb") as f:
    f.write(pipeline_text)
else :
    print("2nd ececution")


# TensorBoardの起動

In [ ]:
%cd $WORK_DIR
%load_ext tensorboard
%tensorboard --logdir=$TRAINED_DIR/train

# 学習の実行
メインイベント  
ちょっと時間がかかるのでお茶でも飲んでてください

In [ ]:
%cd $BASE_DIR/models/research/object_detection/

!python model_main_tf2.py \
--pipeline_config_path=$CONFIG_FILE \
--model_dir=$TRAINED_DIR \
--checkpoint_every_n=500 \
--alsologtostderr

# 回数変更したい場合は --num_train_steps=1000な感じのオプションを追加
# でも、learning_rate も気にしないといけないから pipeline.config を手動で書き換えた方がいいかも

# モデルのエクスポート
モデルをエクスポートしてSavedModelを作成


In [ ]:
%cd $BASE_DIR/models/research/object_detection

if TARGET_DATA == "HAND" :
  EXPORT_DIR_REL="hand_detect"
else:
  EXPORT_DIR_REL="voc_detect"

EXPORT_DIR=os.path.join(WORK_DIR, EXPORT_DIR_REL)

!python exporter_main_v2.py \
--trained_checkpoint_dir=$TRAINED_DIR \
--pipeline_config_path=$CONFIG_FILE \
--output_directory $EXPORT_DIR


エクスポートしたデータをアーカイブ

In [ ]:
%cd $WORK_DIR
# ラベルファイルもコピー
!cp $LABEL_MAP $EXPORT_DIR_REL

# ラベルファイルをテキストファイル化しておく
# ===================================
import sys
import os
import object_detection.utils.label_map_util as label_util

INPUT_LABELS_FILE  = LABEL_MAP
OUTPUT_LABELS_FILE = os.path.join(EXPORT_DIR_REL, "label_map.txt")

category_index = label_util.create_category_index_from_labelmap(INPUT_LABELS_FILE)

with open(OUTPUT_LABELS_FILE, mode='w') as f:
    for i in range(len(category_index)+1) :
        try:
            name = category_index[i]["name"]
        except:
            name = str(i)
        
        # print(name)
        f.write(name + '\n')
# ===================================

import datetime
# 現在時刻(タイムゾーン情報付加)
now = datetime.datetime.now().astimezone(datetime.timezone(datetime.timedelta(hours=+9)))
# ZIPファイル名を生成
zip_filename = now.strftime(f'{EXPORT_DIR_REL}_%Y%m%d_%H%M%S.zip')

!zip -r $zip_filename $EXPORT_DIR_REL

# テスト

In [ ]:
%cd $BASE_DIR
# テスト用画像ファイルのダウンロード
if TARGET_DATA == "HAND" :
  !wget https://cdn.amebaowndme.com/madrid-prd/madrid-web/images/sites/483796/1357355de6edbc4c4b54d22faf0b0756_ce052e9b134a9dbb047a8e17c890832a.jpg -O a.jpg
  !wget https://cdn.amebaowndme.com/madrid-prd/madrid-web/images/sites/483796/564b6ca69e9022aa1977f335a148a05a_2d642c807aaf8f5b972a0a406903447d.jpg -O b.jpg
else :
  !wget https://prtimes.jp/i/6067/298/resize/d6067-298-418042-0.jpg -O a.jpg
  !wget https://www.kic-car.ac.jp/theme/kic_school/img/taisho/ph-society001.jpg -O b.jpg


In [ ]:
import os
import sys
import cv2

import numpy as np
import tensorflow as tf

from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# ラベルマップのロード
PATH_TO_LABELS = LABEL_MAP
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# テスト用イメージファイル
TEST_IMAGE_PATHS = [
                        "a.jpg", 
                        "b.jpg",
                    ]

# モデルのロード
detection_model = tf.saved_model.load(os.path.join(EXPORT_DIR, "saved_model"))

# Check the model's input signature, it expects a batch of 3-color images of type uint8:
print(detection_model.signatures['serving_default'].inputs)

# And returns several outputs:
print(detection_model.signatures['serving_default'].output_dtypes)
print(detection_model.signatures['serving_default'].output_shapes)

# 認識処理関数
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

# 認識処理と表示
def show_inference(model, image_path):
  # 画像の読み込み
  image_np = np.array(Image.open(image_path))
  
  # 認識実行
  output_dict = run_inference_for_single_image(model, image_np)
  
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)
  
  # 表示
  display(Image.fromarray(image_np))
  # ～～～ 単独実行するときの表示処理はこちら ～～～
  # new_image = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
  # cv2.imshow("Detection Results", new_image)  
  # cv2.waitKey(0)
  # cv2.destroyAllWindows()

# 実行
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)

